In [2]:
import sys, re

In [1]:
pip install seaborn

     |████████████████████████████████| 285 kB 571 kB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [6]:
from datetime import date, datetime
from pyspark.sql.types import *
from pyspark.sql.functions import *
import datetime as dt
import pandas as pd
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
import pyspark
from pyspark.sql import SparkSession

In [76]:
def cal_std_day(befor_day):   
    x = dt.datetime.now() - dt.timedelta(befor_day)
    year = x.year
    month = x.month if x.month >= 10 else '0'+ str(x.month)
    day = x.day if x.day >= 10 else '0'+ str(x.day)  
    return str(year)+ '-' +str(month)+ '-' +str(day)

In [11]:
# from pyspark.sql.session import SparkSession

# spark = SparkSession.builder.getOrCreate()

In [13]:
from pyspark import SparkContext
sc = SparkContext.getOrCreate();

In [73]:
path = '/news_data/article'
co_patient_json = spark.read.json(path, encoding='UTF-8')
co_patient_json.show()

+-------------------------------+------------------------------+
|                           data|                          meta|
+-------------------------------+------------------------------+
|   [{"상장사, 4분기 더 강한 ...|{{뉴스제목}, 네이버 실시간 ...|
|[{코스피, 영란은행이 만든 반...|{{뉴스제목}, 네이버 실시간 ...|
+-------------------------------+------------------------------+



In [79]:
#증권사 이름 빼기
stopwords=['하이투자증권','삼정kpmg','EY한영','등','미래에셋증권','출발','가능성','끝모를','소식에','장중','상장','美','기관','달러','경기','체결','위한','더','후','title','코스피','08','코스닥','규모','상승','하락','0','1','2','3','4','원','혼신의','마감','증시','환율은','살','환율은','제출']

In [80]:

word_count=sc.textFile('/news_data/article/realtimenews_' + cal_std_day(0) + '.json') \
                .filter(lambda e: len(e) > 0) \
                .flatMap(lambda e:re.split('\W+', e)) \
                .filter(lambda x:x.lower() not in stopwords) \
                .filter(lambda e: len(e) > 0) \
                .map(lambda e : (e.lower(), 1)) \
                .reduceByKey(lambda a,b :a+b) \
                .sortBy(lambda e:e[1],ascending=False)
word_count
word_count2=word_count.filter(lambda x:x!="('title',381)")
# type(word_count)
# word_count2=word_count.str.strip("title")                
#'\W+':문자가 아닌걸 다 나누는 기준점으로 삼겠다.
#  .map(lambda e :(e.ower(), 1) ): 영단어가 key, count가 value 라서 1 넣어주기=> 각 단어,1 조합=>추후 reducebykey로 다 더하기 위해
#Lower : 소문자로 바꿔줌
word_count2.collect()

[('연저점', 39),
 ('출발', 34),
 ('특징주', 31),
 ('경신', 28),
 ('삼성전자', 22),
 ('신저가', 20),
 ('환율', 18),
 ('또', 14),
 ('52주', 14),
 ('속보', 11),
 ('45', 11),
 ('11', 11),
 ('첫날', 10),
 ('개최', 10),
 ('출발하는', 10),
 ('거래', 10),
 ('더블유씨피', 10),
 ('애플', 10),
 ('내린', 9),
 ('등', 9),
 ('급락', 9),
 ('2161', 9),
 ('매출', 8),
 ('esg', 8),
 ('추락', 8),
 ('장', 8),
 ('네이버', 8),
 ('주식', 8),
 ('해제', 8),
 ('자사주', 8),
 ('공모가', 8),
 ('초반', 8),
 ('목표가', 7),
 ('etf', 7),
 ('26', 7),
 ('개발', 7),
 ('미래에셋증권', 7),
 ('10월', 7),
 ('서비스', 7),
 ('경신한', 7),
 ('실적', 7),
 ('강세', 6),
 ('의무보유', 6),
 ('wcp', 6),
 ('연준', 6),
 ('주가', 6),
 ('케이카', 6),
 ('첫', 6),
 ('붕괴', 6),
 ('내달', 6),
 ('개인', 6),
 ('하락한', 6),
 ('약세', 6),
 ('소폭', 6),
 ('기대', 5),
 ('대비', 5),
 ('계약', 5),
 ('9시부터', 5),
 ('만에', 5),
 ('쏘카', 5),
 ('ey한영', 5),
 ('확대', 5),
 ('진행', 5),
 ('대', 5),
 ('개장', 5),
 ('9', 5),
 ('668', 5),
 ('금리', 5),
 ('42개사', 5),
 ('하향', 5),
 ('동경거래소', 5),
 ('대어', 5),
 ('돌파', 5),
 ('오전', 5),
 ('준비', 5),
 ('전', 5),
 ('상장예비심사', 5),
 ('시황', 5),
 ('하이투자증

In [30]:
print(word_count.take(10))

[('title', 381), ('코스피', 58), ('마감', 54), ('1', 29), ('환율', 28), ('코스닥', 27), ('규모', 27), ('권순우', 21), ('하락', 20), ('상승', 19)]


In [16]:
# pip install wordcloud

Note: you may need to restart the kernel to use updated packages.


In [3]:
font_path = 'malgun.ttf'

In [6]:
from wordcloud import WordCloud, STOPWORDS
wc = WordCloud(background_color='white',width=800, height =400,max_font_size=80,font_path =font_path) 
cloud = wc.generate_from_frequencies(dict(word_count.collect()))
dict(word_count.collect()) #확인

cloud.to_file('/home/big/project/wc_news_'+cal_std_day(0)+'.jpg') #파일로 생성, matplotlib으로 생성하는것도 있음



NameError: name 'word_count' is not defined